In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import joblib

# Load the dataset
file_path = 'Real estate.csv'
data = pd.read_csv(file_path)

# Display first few rows to understand the structure
print(data.head())

# Assuming the target variable is 'Price' (replace if different)
# Selecting features and target (modify columns accordingly)
X = data.drop(columns=['Y house price of unit area'])  # Features
y = data['Y house price of unit area']  # Target

# Splitting the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling (if necessary, assuming continuous variables)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize the Bagging Regressor model
model = BaggingRegressor(n_estimators=50, random_state=42)

# Train the model
model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test_scaled)

# Evaluate the model performance
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Save the model and scaler for later use in the web application
joblib.dump(model, 'bagging_regressor_model.pkl')
joblib.dump(scaler, 'scaler.pkl')


   No  X1 transaction date  X2 house age  \
0   1             2012.917          32.0   
1   2             2012.917          19.5   
2   3             2013.583          13.3   
3   4             2013.500          13.3   
4   5             2012.833           5.0   

   X3 distance to the nearest MRT station  X4 number of convenience stores  \
0                                84.87882                               10   
1                               306.59470                                9   
2                               561.98450                                5   
3                               561.98450                                5   
4                               390.56840                                5   

   X5 latitude  X6 longitude  Y house price of unit area  
0     24.98298     121.54024                        37.9  
1     24.98034     121.53951                        42.2  
2     24.98746     121.54391                        47.3  
3     24.98746     121.54391  

['scaler.pkl']

In [5]:
from flask import Flask, request, jsonify, render_template
import joblib
import numpy as np

# Load the trained model and scaler
model = joblib.load('bagging_regressor_model.pkl')
scaler = joblib.load('scaler.pkl')

app = Flask(__name__)

# Home page
@app.route('/')
def home():
    return render_template('index.html')  # Your 'index.html' form already exists

# Prediction page
@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Get data from the form (ensure form fields correspond to your feature names)
        features = [float(x) for x in request.form.values()]
        features_array = np.array([features])

        # Scale the input
        features_scaled = scaler.transform(features_array)

        # Predict using the trained model
        prediction = model.predict(features_scaled)

        # Return result to the page
        return render_template('index.html', prediction_text=f'Predicted Price: ${prediction[0]:.2f}')

    except Exception as e:
        # Return an error message if something goes wrong
        return render_template('index.html', prediction_text=f"Error occurred: {str(e)}")

if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [20/Oct/2024 18:24:03] "GET / HTTP/1.1" 500 -
Traceback (most recent call last):
  File "c:\Users\tushu\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 1498, in __call__
    return self.wsgi_app(environ, start_response)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\tushu\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 1476, in wsgi_app
    response = self.handle_exception(e)
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\tushu\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\tushu\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Use